In [ ]:
!rm -rf /content/sample_data

In [ ]:
!rm -rf /content/suichang_round1_train_210120
!rm -rf /content/suichang_round1_train_210120.zip
!cp -r /content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/suichang_round1_train_210120.zip ./
!unzip -q suichang_round1_train_210120.zip

In [ ]:
!rm -rf /content/suichang_round1_train_210120.zip

In [ ]:
!pip install -U albumentations

In [ ]:
import albumentations as A
import os.path as osp
import numpy as np
import pandas as pd
import pathlib, sys, os, random, time, glob
import numba, cv2, gc
from tqdm import tqdm
import torch.utils.data as D
import torchvision
from torchvision import transforms as T
from albumentations.pytorch import ToTensorV2

In [ ]:
IMAGE_SIZE =256
class TianChiDataset(D.Dataset):
    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform

        self.len = len(paths)
        self.as_tensor = T.Compose([
            T.ToPILImage(),
            T.Resize(IMAGE_SIZE),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

    # get data operation
    def __getitem__(self, index):
        img = cv2.imread(self.paths[index])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.paths[index].replace('.tif', '.png'),cv2.IMREAD_UNCHANGED) #- 1
        mask = cv2.resize(mask, (IMAGE_SIZE, IMAGE_SIZE))
        transformed = self.transform(image=img, mask=mask)
        image = transformed["image"]
        mask = transformed["mask"]
        return image, mask

In [ ]:
train_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        A.RandomSizedCrop([128, 256],256,256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
dataset_path = './'
dataset = TianChiDataset(glob.glob(osp.join(dataset_path, 'suichang_round1_train_210120', '*.tif')),train_transform)
print(len(dataset))

In [ ]:
import copy
import matplotlib.pyplot as plt
def visualize_augmentations(dataset, idx=0, samples=8):
    dataset = copy.deepcopy(dataset)
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    figure, ax = plt.subplots(nrows=samples, ncols=2, figsize=(10, 24))
    for i in range(samples):
        image, mask = dataset[idx]
        ax[i, 0].imshow(image)
        ax[i, 1].imshow(mask, interpolation="nearest")
        ax[i, 0].set_title("Augmented image")
        ax[i, 1].set_title("Augmented mask")
        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
random.seed(42)
visualize_augmentations(dataset, idx=55)

In [ ]:
!rm -rf dataug
!mkdir dataug

In [ ]:
import PIL
from PIL import Image, ImageChops, ImageEnhance
def save_augmentations(dataset, idx=0, samples=8):
    dataset = copy.deepcopy(dataset)
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    for i in range(samples):
        image, mask = dataset[idx]
        image=Image.fromarray(image)
        mask=Image.fromarray(mask)
        image.save('dataug/aug_{}_{}.tif'.format(i,idx))
        mask.save('dataug/aug_{}_{}.png'.format(i,idx))

In [ ]:
for id in tqdm(range(len(dataset))):
    save_augmentations(dataset, idx=id, samples=8)

100%|██████████| 16017/16017 [15:35<00:00, 17.12it/s]


In [ ]:
image_file_number=glob.glob(pathname='/content/dataug/*.tif')
mask_file_number=glob.glob(pathname='/content/dataug/*.png')
print(len(image_file_number),len(mask_file_number))

128136 128136


In [ ]:
!zip -r -q dataug.zip dataug
!cp -r dataug.zip /content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛